In [1]:
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
from pathlib import Path

In [4]:
import json

In [5]:
import numpy as np

In [6]:
import pandas as pd

In [7]:
from sklearn.pipeline import Pipeline

In [8]:
from sklearn.compose import ColumnTransformer

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [10]:
from sklearn.impute import SimpleImputer

In [12]:
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    precision_recall_curve,
    roc_curve
)

In [13]:
from xgboost import XGBClassifier

In [14]:
RANDOM_STATE = 42

In [15]:
PROJECT_ROOT = Path('.')

In [16]:
FEATURES_DIR = PROJECT_ROOT / 'data' / 'features'

In [17]:
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'

In [18]:
ARTIFACTS_DIR.mkdir(parents=True, exist_ok = True)

In [19]:
TARGET_COL = 'revenue'

In [20]:
train_path = FEATURES_DIR / 'train_safe_fe.csv'

In [21]:
valid_path = FEATURES_DIR / 'valid_safe_fe.csv'

In [22]:
test_path = FEATURES_DIR / 'test_safe_fe.csv'

In [23]:
train_df = pd.read_csv(train_path)

In [24]:
valid_df = pd.read_csv(valid_path)

In [25]:
train_df.shape, valid_df.shape

((8543, 31), (2563, 31))

In [26]:
def split_xy(df:pd.DataFrame, target_col:str = TARGET_COL):
    X = df.drop(columns = [target_col])
    y = df[target_col].astype(int)
    return X,y

In [27]:
X_train, y_train = split_xy(train_df)

In [28]:
X_valid, y_valid = split_xy(valid_df)

In [29]:
X_train.shape, X_valid.shape, y_train.mean(), y_valid.mean()

((8543, 30),
 (2563, 30),
 np.float64(0.1563853447266768),
 np.float64(0.15606710885680844))

In [30]:
def build_preprocess_pipeline(X:pd.DataFrame):
    cat_cols = X.select_dtypes(include = ['object']).columns.tolist()
    num_cols = [c for c in X.columns if c not in cat_cols]
    numeric_transformer = Pipeline(steps =[
        ('imputer', SimpleImputer(strategy = 'median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps = [
        ('imputer', SimpleImputer(strategy = 'most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ])
    preprocessor = ColumnTransformer (
        transformers = [
            ('num', numeric_transformer,num_cols),
            ('cat', categorical_transformer,cat_cols)
        ],
        remainder = 'drop'
    )
    return preprocessor, num_cols, cat_cols

In [31]:
preprocess, num_cols, cat_cols = build_preprocess_pipeline(X_train)

In [32]:
len(num_cols), cat_cols

(28, ['month', 'visitor_type'])

In [34]:
xgb = Pipeline(steps = [
    ('prep', preprocess),
    ('model', XGBClassifier(
        n_estimators = 500,
        learning_rate = 0.05,
        max_depth = 4,
        subsample = 0.9,
        colsample_bytree = 0.9,
        reg_lambda = 1.0,
        random_state = RANDOM_STATE,
        eval_metric = 'logloss'
    ))
])

In [35]:
xgb.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('prep', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains s

In [36]:
proba_valid = xgb.predict_proba(X_valid)[:,1]

In [37]:
pred_valid_05 = (proba_valid >= 0.50).astype(int)

In [38]:
metrics_05 = {
    'roc_auc': roc_auc_score(y_valid,proba_valid),
    'precision@0.50': precision_score(y_valid,pred_valid_05,zero_division =0),
    'recall@0.50': recall_score(y_valid,pred_valid_05, zero_division = 0),
    'f1@0.50': f1_score(y_valid,pred_valid_05,zero_division = 0),
}

In [39]:
metrics_05

{'roc_auc': 0.7806426259824318,
 'precision@0.50': 0.40625,
 'recall@0.50': 0.0975,
 'f1@0.50': 0.15725806451612903}

In [40]:
cm_05 = confusion_matrix(y_valid,pred_valid_05)

In [41]:
cm_05

array([[2106,   57],
       [ 361,   39]])

In [43]:
print('Confusion Matrix @0.50')
print(cm_05)
print('\nClassification Report @0.50')
print(classification_report(y_valid,pred_valid_05,digits=4, zero_division = 0))

Confusion Matrix @0.50
[[2106   57]
 [ 361   39]]

Classification Report @0.50
              precision    recall  f1-score   support

           0     0.8537    0.9736    0.9097      2163
           1     0.4062    0.0975    0.1573       400

    accuracy                         0.8369      2563
   macro avg     0.6300    0.5356    0.5335      2563
weighted avg     0.7838    0.8369    0.7923      2563



In [52]:
def threshold_table(y_true,proba,thresholds):
    rows = []
    for t in thresholds:
        pred = (proba >=t).astype(int)
        rows.append({
            'threshold':float(t),
            'precision': precision_score(y_true, pred, zero_division = 0),
            'recall': recall_score(y_true, pred, zero_division = 0),
            'f1': f1_score(y_true,pred,zero_division = 0),
        })
    return pd.DataFrame(rows)

In [53]:
thresholds = np.arange(0.05,0.96,0.05)

In [54]:
thr_df = threshold_table(y_valid,proba_valid,thresholds)

In [55]:
thr_df.sort_values('f1', ascending = False).head(10)

,threshold,precision,recall,f1
3,0.20,0.337094,0.5975,0.431019
2,0.15,0.303347,0.7250,0.427729
4,0.25,0.363803,0.4975,0.420275
1,0.10,0.266041,0.8500,0.405244
5,0.30,0.394231,0.4100,0.401961
6,0.35,0.444444,0.3500,0.391608
0,0.05,0.219583,0.9475,0.356538
7,0.40,0.475113,0.2625,0.338164
8,0.45,0.459119,0.1825,0.261181
9,0.50,0.406250,0.0975,0.157258


In [56]:
thr_df.sort_values(['recall','precision'],ascending = [False,False])

,threshold,precision,recall,f1
0,0.05,0.219583,0.9475,0.356538
1,0.10,0.266041,0.8500,0.405244
2,0.15,0.303347,0.7250,0.427729
3,0.20,0.337094,0.5975,0.431019
4,0.25,0.363803,0.4975,0.420275
5,0.30,0.394231,0.4100,0.401961
6,0.35,0.444444,0.3500,0.391608
7,0.40,0.475113,0.2625,0.338164
8,0.45,0.459119,0.1825,0.261181
9,0.50,0.406250,0.0975,0.157258


In [57]:
FP_COST = 1.0

In [58]:
FN_COST = 5.0

In [59]:
def expected_cost(y_true,proba,threshold,fp_cost = FP_COST, fn_cost = FN_COST):
    pred = (proba >= threshold).astype(int)
    tn,fp,fn,tp = confusion_matrix(y_true,pred).ravel()
    cost = fp*fp_cost + fn * fn_cost
    return cost, {'tn':tn, 'fp':fp, 'fn':fn, 'tp':tp}

In [60]:
cost_rows = []

In [66]:
for t in thresholds:
    cost, parts = expected_cost(y_valid,proba_valid,t)
    cost_rows.append({
        'threshold':float(t),
        'expected_cost':float(cost),
        **parts
    })

In [67]:
cost_df = pd.DataFrame(cost_rows).sort_values('expected_cost')

In [68]:
cost_df.head(10)

,threshold,expected_cost,tn,fp,fn,tp
2,0.15,1216.0,1497,666,110,290
1,0.10,1238.0,1225,938,60,340
3,0.20,1275.0,1693,470,161,239
4,0.25,1353.0,1815,348,201,199
5,0.30,1432.0,1911,252,236,164
0,0.05,1452.0,816,1347,21,379
6,0.35,1475.0,1988,175,260,140
7,0.40,1591.0,2047,116,295,105
8,0.45,1721.0,2077,86,327,73
9,0.50,1862.0,2106,57,361,39


In [70]:
best_cost_threshold = float(cost_df.iloc[0]['threshold'])

In [71]:
best_cost_threshold

0.15000000000000002

In [72]:
t_star = best_cost_threshold

In [73]:
pred_valid_star = (proba_valid >= t_star).astype(int)

In [77]:
metrics_star = {
    'threshold':t_star,
    'roc_auc': roc_auc_score(y_valid,proba_valid),
    'precision': precision_score(y_valid,pred_valid_star, zero_division = 0),
    'recall': recall_score(y_valid,pred_valid_star, zero_division = 0),
    'f1': f1_score(y_valid,pred_valid_star,zero_division = 0),
}

In [78]:
cm_star = confusion_matrix(y_valid,pred_valid_star)

In [79]:
metrics_star, cm_star

({'threshold': 0.15000000000000002,
  'roc_auc': 0.7806426259824318,
  'precision': 0.303347280334728,
  'recall': 0.725,
  'f1': 0.4277286135693215},
 array([[1497,  666],
        [ 110,  290]]))

In [80]:
print(f"Chosen threshold (cost_sensitive): {t_star:.2f}")
print('Confusion Matrix')
print(cm_star)
print('\nClassification Report')
print(classification_report(y_valid, pred_valid_star, zero_division = 0))

Chosen threshold (cost_sensitive): 0.15
Confusion Matrix
[[1497  666]
 [ 110  290]]

Classification Report
              precision    recall  f1-score   support

           0       0.93      0.69      0.79      2163
           1       0.30      0.72      0.43       400

    accuracy                           0.70      2563
   macro avg       0.62      0.71      0.61      2563
weighted avg       0.83      0.70      0.74      2563



In [81]:
##hedefimiz recall = 0.70 gibi varsayımda yapılacak işlemler:

In [82]:
precision_arr, recall_arr,thr_arr = precision_recall_curve(y_valid,proba_valid)

In [83]:
pr_df = pd.DataFrame({
    'threshold': np.append(thr_arr, 1.0),
    'precision':precision_arr,
    'recall': recall_arr
    
})

In [84]:
target_recall = 0.70

In [86]:
candidates = pr_df[pr_df['recall'] >= target_recall].sort_values('precision', ascending = False)

In [87]:
candidates.head(10)

,threshold,precision,recall
1671,0.163593,0.315022,0.7025
1670,0.163361,0.314670,0.7025
1669,0.162881,0.314318,0.7025
1672,0.163654,0.314254,0.7000
1668,0.162771,0.313966,0.7025
1667,0.162635,0.313616,0.7025
1666,0.162431,0.313266,0.7025
1665,0.162371,0.312918,0.7025
1664,0.161775,0.312570,0.7025
1663,0.161740,0.312222,0.7025


In [88]:
decision = {
    'model':'XGBoost (SAFE feature set)',
    'threshold_default': 0.50,
    'threshold_cost_sensitive': t_star,
    'cost_assumptions': {'FP_COST':FP_COST, 'FN_COST': FN_COST},
    'valid_metrics_default_0.50': metrics_05,
    'valid_metrics_cost_sensitive':metrics_star,
}

In [89]:
out_path = ARTIFACTS_DIR / 'decision_threshold_day26.json'

In [90]:
with open(out_path,'w', encoding = 'utf-8') as f:
    json.dump(decision,f,ensure_ascii = False, indent = 2)

In [ ]:
out_path

## Business Insights & Decision Rationale (Day 26)

### 1. Model Performance in Business Context
The selected XGBoost model demonstrates a strong ability to rank users by purchase probability (ROC-AUC ≈ 0.78 on the validation set).  
However, at the default threshold (0.50), the model prioritizes precision over recall, resulting in a large number of false negatives. In an e-commerce context, this means that many potential buyers would be missed.

From a business perspective, this behavior is suboptimal when the primary objective is to maximize conversions rather than strictly control marketing spend.

---

### 2. Threshold Selection and Cost-Sensitive Evaluation
Instead of relying on the default decision threshold (0.50), we performed threshold tuning and evaluated the trade-off between false positives and false negatives.

We introduced a cost-sensitive perspective with the following assumptions:
- False Positive (FP): unnecessary campaign or incentive shown to a user (lower cost)
- False Negative (FN): missed potential buyer (higher opportunity cost)

Using illustrative costs (FP = 1, FN = 5), the threshold that minimizes the expected business cost was **0.15**.

This threshold significantly increases recall, ensuring that a larger share of potential buyers is identified, at the expense of a higher number of false positives. Given the relative costs, this trade-off is justified for a conversion-oriented strategy.

---

### 3. Business Interpretation of Errors
- **False Positives** represent users who receive a campaign but do not convert. These cases increase marketing spend but do not directly harm revenue.
- **False Negatives** represent users who were likely to purchase but were not targeted. These cases result in lost revenue opportunities.

In most e-commerce scenarios, the cost of missing a potential buyer outweighs the cost of an unnecessary campaign, making recall-focused optimization a more appropriate objective.

---

### 4. Recommended Deployment Strategy
The model should be used as a **ranking and prioritization tool**, not a binary decision-maker.

Recommended usage:
- Score all active users with a predicted purchase probability.
- Target the top segment (e.g., top 15–25%) for campaigns such as discounts, emails, or personalized offers.
- Adjust the decision threshold dynamically based on campaign budget and seasonality.

This approach allows marketing teams to balance conversion maximization and cost control without retraining the model.

---

### 5. Limitations and Next Steps
- The cost assumptions used in this analysis are illustrative and should be calibrated using real campaign costs and customer lifetime value.
- Future improvements could include:
  - Dynamic thresholding by user segment or season
  - Incorporating real marketing cost data into the objective function
  - Online A/B testing to validate the business impact of threshold changes

Overall, the model provides a strong foundation for data-driven decision-making in conversion-focused e-commerce environments.
